In [1]:
import pandas as pd
import numpy as np

In [2]:
#10 year bond rate from yahoo finance
rf = 0.01732

In [3]:
#bloomberg sample data for 5 currencies 
df = pd.read_excel('Dummy Dataset.xlsx')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
product_list = list(df.columns[1:len(df.columns)])

# Sharpe Ratio

In [4]:
#add returns to the bloomberg datasets 
def preprocessing(df):
    df['month'] = pd.DatetimeIndex(df['Date']).month
    j=2
    for i in product_list:
        col1 = i+'_prev'
        col2=i+'_return'
        df.insert(j,col1,df[i].shift(1))
        df.insert(j+1,col2,df[i]/df[col1]-1)
        j+=3
    return df
preprocessing(df)
df.head()

,Date,KWN,KWN_prev,KWN_return,KRW,KRW_prev,KRW_return,TWD,TWD_prev,TWD_return,NTN,NTN_prev,NTN_return,IRN,IRN_prev,IRN_return,month
0,2019-09-17,1188.56,NaN,NaN,1189.70,NaN,NaN,30.972,NaN,NaN,30.826,NaN,NaN,70.04,NaN,NaN,9
1,2019-09-16,1184.18,1188.56,-0.003685,1183.15,1189.70,-0.005506,30.983,30.972,0.000355,30.777,30.826,-0.001590,69.98,70.04,-0.000857,9
2,2019-09-13,1178.69,1184.18,-0.004636,1190.80,1183.15,0.006466,31.043,30.983,0.001937,30.762,30.777,-0.000487,70.07,69.98,0.001286,9
3,2019-09-12,1181.72,1178.69,0.002571,1190.80,1190.80,0.000000,31.043,31.043,0.000000,30.762,30.762,0.000000,69.89,70.07,-0.002569,9
4,2019-09-11,1189.65,1181.72,0.006711,1190.80,1190.80,0.000000,31.211,31.043,0.005412,31.027,30.762,0.008615,69.74,69.89,-0.002146,9


In [5]:
#to calculate the expected return and volatility for each currency 
#it returns a dictionary 
def basic_measures(df):
    s = dict()
    for i in product_list:
        s[i]=dict()
        j = i+'_return'
        
        #annualized expected return
        expected_return = df[j].mean()*252
        s[i]['expected_return'] = expected_return
        
        #standard deviation 
        sd = df[j].std()
        s[i]['standard_deviation'] = sd 
        
    return s  

basic_measures(df)

{'IRN': {'expected_return': 0.04699204937729283,
  'standard_deviation': 0.0038542972903242625},
 'KRW': {'expected_return': 0.016561093955133353,
  'standard_deviation': 0.004232137783714444},
 'KWN': {'expected_return': 0.01403019088287639,
  'standard_deviation': 0.004143548614460702},
 'NTN': {'expected_return': 0.0605607736547208,
  'standard_deviation': 0.003356061286617935},
 'TWD': {'expected_return': 0.029266063431462326,
  'standard_deviation': 0.0021983595326922375}}

In [6]:
#dummy transation data
transation_df = pd.read_csv('Transactions.csv')


In [7]:
def trans_preprocessing():
    #handle direction and notional of trade 
    #Size is a new column with the direction of trade
    # long: positive; short: negative
    transation_df['Size'] = transation_df.Direction.apply(
               lambda x: (1 if x == 'long' else -1))
    transation_df['Size'] = transation_df.Size * transation_df['Size/Notional']

In [8]:
trans_preprocessing()
user_list = transation_df['User'].unique()
user_list

array(['A', 'B'], dtype=object)

In [9]:
def portfolio_weight(pf):
    df = pd.pivot_table(pf, values='Size',columns='Product',aggfunc='sum')
    total = df.sum(axis=1)
    s= dict()
    for i in product_list:
        if i in df.columns:
            s[i]= float(df[i]/total)
        else:
            s[i]= 0
    return s

# create the correlation matrix
def cov(df):
    ls = []
    for i in product_list:
        col = i+'_return'
        ls.append(col)
    returns = df[ls]
    cov_matrix = returns.cov()*252
    return cov_matrix

def portfolio_volatility(pf):
    w = np.array(list(portfolio_weight(pf).values()))
    #Annualized sd
    volatility = np.sqrt(w.T.dot(cov(df)).dot(w))
    return volatility


measures = basic_measures(df)
def portfolio_return(df):
    weights = portfolio_weight(df)
    portfolio_return = sum(measures[k]['expected_return']*weights[k] for k in measures)
    return portfolio_return

def sharpe_ratio(df):
    sr = (portfolio_return(df) - rf)/portfolio_volatility(df)
    return sr

In [10]:
portfolio_list = transation_df['Portfolio'].unique()

def main_sharpe(pf_list):
    s = dict()
    for i in pf_list:
        pf = transation_df.groupby('Portfolio').get_group(i)
        s[i] = sharpe_ratio(pf)
    return s
main_sharpe(portfolio_list)

{'A1': 0.22809541601147856,
 'A2': 0.6359791760701712,
 'B1': 0.046527347436494844}

# Sortino Ratio

In [11]:
#set daily MAR value
MAR = 0.0005

In [12]:
#add returns to the bloomberg datasets 
def preprocessing2(df):
    x=4
    for i in range(len(product_list)):
        col1 = product_list[i]+'_return'
        col2 = product_list[i]+'_dev'
        ls = []
        i +=1
        for j in range(df.shape[0]):
            val = df[col1][j]
            if  val < MAR:
                dev = (MAR-val)**2
                ls.append(dev)
            else:
                ls.append(0)          
        df.insert(x,col2,ls)
        x+=4
        
    return df


In [13]:
def cal_month_return(df):
    dic_result = {}
    months = df['month'].unique()
    dic_result['month'] = months
    for j in product_list:
        col1 = j+"_return"
        dic_result[col1] = []
        for i in months:
            dic_result[col1].append(df.groupby('month').get_group(i)[col1].mean())
    return pd.DataFrame(dic_result)

In [14]:
cal_month_return(df)

,month,KWN_return,KRW_return,TWD_return,NTN_return,IRN_return
0,9,0.001758,0.001607,0.001201,0.001337,0.000652
1,8,-0.000635,-0.000831,-0.000298,0.000066,-0.000414
2,7,-0.001244,-0.001099,-0.000251,-0.000493,-0.000153
3,6,0.000776,0.001010,0.000809,0.000635,0.001198
4,5,0.001144,0.001035,-0.000632,0.000307,-0.000207


# PnL

In [15]:
curr_price = {}
x = 1
for i in product_list:
    curr_price[i] = df.iloc[0,x]
    x += 3

In [16]:
#input portfolio_list
#output dict: {'portfolio_ID':{'product1':profit}}
def cal_profit_by_product(portfolios):
    dic_result = {}
    for i in portfolios:
        temp = {}
        pf = transation_df.groupby('Portfolio').get_group(i)
        products = pf['Product'].unique()
        for j in products:
            pf_product = pf.groupby('Product').get_group(j)
            temp[j] = ((curr_price[j] - pf_product['Price']) * pf_product['Size']).sum()
        dic_result[i] = temp
    return dic_result
        
        

In [17]:
cal_profit_by_product(portfolio_list)

{'A1': {'IRN': 6.300000000000168, 'KWN': 162.04999999999927},
 'A2': {'NTN': 7.1320000000000086, 'TWD': -0.011999999999986244},
 'B1': {'IRN': 4.97999999999999, 'KRW': 78.10000000000059}}

# Hit Ratio (win-loss ratio)

In [18]:
def win_loss(users):
    dict_result = {}
    for i in users:
        user = transation_df.groupby('User').get_group(i) 
        a = 0
        b = 0
        for j in range(user.shape[0]):
            currency = user.iloc[j,2]
            p = user.iloc[j,4]
            s = user.iloc[j,12]
            profit = (curr_price[currency]-p)*s
            if profit > 0:
                a += 1
            else:
                b += 1
        if b == 0:
            ratio = a
        else:
            ratio = a/b
        dict_result[i] = ratio
    return dict_result
        
    

In [19]:
win_loss(user_list)

{'A': 1.6, 'B': 5}